### Importy

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tmd

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload
import DataLoader.xAPIConnector
reload(DataLoader.xAPIConnector)
from DataLoader.xAPIConnector import *


import DataLoader.DataLoader
reload(DataLoader.DataLoader)
from DataLoader.DataLoader import *

from DataLoader.config import user_id, pwd

# Pobieranie danych

Należy podać listę symboli, datę, od której chcemy zaciągnąć dane i częstotliwość (teraz '5min'). Można też podać datę 'end', ale domyślnie zaciąga się do chwili obecnej.

In [16]:
symbols = ['BITCOIN', 'ETHEREUM']
start, interval = '2024-12-01 00:00:00', '5min'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

[2025-01-14 11:46:43.372569] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-14 11:46:50.603566] Wylogowuję z API...


Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [106]:
window = int(60/int(interval[:-3]))
returnsBTC = data.loc[:, 'BITCOIN'].pct_change().dropna()

In [141]:
def prepareData(data: pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window))
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.values
        
        i = i - skip    
        count += 1
        
    return np.array(X)

# Modele klasyfikacyjne

In [156]:
from sklearn.linear_model import LogisticRegression as LogR, LinearRegression as LinR
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import GradientBoostingClassifier as GBC

### Definiowanie klas

Pierwszy pomysł na zdefiniowanie klas to przyjęcie $1$ jeśli stopa zwrotu jest $>$'threshold' oraz $0$ w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > $0$" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [135]:
skip = 1
arr = prepareData(data=returnsBTC, window=window, skip=skip)
X = arr[:, :-1]
y = arr[:, -1]

threshold = np.quantile(y, 0.75)
y = (y > threshold)*1.0

for i in range(0, 2):
    print(f"Klasa {i}: {np.sum(y == i)/len(y):.4%}")

Klasa 0: 75.0027%
Klasa 1: 24.9973%


Teraz podzielimy dane na treningowe i testowe, po czym sprawdzimy jakość predykcji

In [136]:
train_test_ratio = 0.7
len_train = int(train_test_ratio*X.shape[0])

Xtrain = X[:len_train, :]
ytrain = y[:len_train]

Xtest = X[len_train:, :]
ytest = y[len_train:]

print(f"[INFO] W zbiorze treningowym znaduje się {Xtrain.shape[0]} obserwacji.")
print(f"[INFO] W zbiorze testowym znajduje się {Xtest.shape[0]} obserwacji.")

[INFO] W zbiorze treningowym znaduje się 6527 obserwacji.
[INFO] W zbiorze testowym znajduje się 2798 obserwacji.


In [ ]:
print(f"[INFO] Regresja logistyczna")
clf = LogR().fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Regresja logistyczna
	Skuteczność treningowa modelu: 76.3138%
	Skuteczność testowa modelu: 71.9442%


In [138]:
print(f"[INFO] Support Vector Machines")
clf = SVC(kernel='poly').fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Support Vector Machines
	Skuteczność treningowa modelu: 77.9838%
	Skuteczność testowa modelu: 70.8006%


In [139]:
print(f"[INFO] Drzewo decyzyjne")
clf = DTC(max_depth=3).fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Drzewo decyzyjne
	Skuteczność treningowa modelu: 76.6662%
	Skuteczność testowa modelu: 71.8013%


In [140]:
print(f"[INFO] Las decyzyjny")
clf = GBC(n_estimators=100, max_depth=5).fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Las decyzyjny
	Skuteczność treningowa modelu: 83.9896%
	Skuteczność testowa modelu: 69.0136%


Predykcje uzyskane przy użyciu tych metod tworzenia $y$ dają rezultaty gorsze od naiwnego klasyfikatora przypisującego zawsze $0$.

# Model na akcjach spółek surowcowych

## Strategia trend-following

In [2]:
def position(x):
    if x[1] == True: return 1
    elif x[0] == True and x[1] == False: return -1
    elif x[0] == False and x[1] == False: return 0
    
def EMA(y: pd.Series, alpha: float):
    weights = np.array([(1-alpha)**(len(y)-i+1) for i in range(len(y))])
    return (y.values).dot(weights)/np.sum(weights)

In [3]:
import json

with open('spolki.json') as f:
    data = json.load(f)

In [4]:
gold_symbols = [key+'.US' for key in data['gold'].keys()]
gold_symbols

['NEM.US',
 'AU.US',
 'RGLD.US',
 'CDE.US',
 'SA.US',
 'CMCL.US',
 'IDR.US',
 'CTGO.US',
 'FTCO.US',
 'USAU.US']

In [5]:
symbols = gold_symbols
start, interval = '2020-01-01 00:00:00', '1D'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

[2025-01-28 11:38:17.463928] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-28 11:38:18.163547] Wylogowuję z API...
[2025-01-28 11:38:18.345255] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-28 11:38:18.894155] Wylogowuję z API...
[2025-01-28 11:38:19.011485] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-28 11:38:19.676766] Wylogowuję z API...
[2025-01-28 11:38:19.777325] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano CDE.US
[2025-01-28 11:38:20.248875] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano SA.US
[2025-01-28 11:38:20.827652] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano CMCL.US
[2025-01-28 11:38:21.310400] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano IDR.US
[2025-01-28 11:38:21.

In [6]:
data

,NEM.US,AU.US,RGLD.US
Date,,,
2020-01-02,43.25,22.64,120.80
2020-01-03,42.73,22.49,119.94
2020-01-06,43.21,22.62,118.32
2020-01-07,43.24,22.43,120.01
2020-01-08,41.98,21.44,111.02
...,...,...,...
2025-01-21,42.18,27.83,140.33
2025-01-22,41.64,27.94,138.37
2025-01-23,41.70,27.82,138.26


In [75]:
from scipy.stats import pearsonr
def cValue(y: pd.Series):
    n = len(y)
    c = np.array(range(1, n+1))
    corr, p = pearsonr(c, np.array(y.values))
    if p > 0.05: corr = 0.0 # korelacja jest pozorna
    return corr

def RSI(y: pd.Series):
    U = y[y>0].sum()
    D = -y[y<0].sum()
    RS = U/D
    rsi = 100-100/(1+RS)
    return rsi

def RSIPosition(x: pd.Series, decision: str):
    if decision == 'buy':
        pass
    elif decision == 'sell':
        pass
    elif decision == 'wait':
        pass

valid_methods = ['ma', 'rsi', 'macd', 'cvalue']
def Decision(y: pd.Series,
             method: str = 'MACD',
             short_window = 12,
             long_window = 26,
             signal_window = 9):
    method=method.lower()
    assert method in valid_methods, "[BŁĄD] Podana metoda nie jest zaimplementowana."
    
    if method == 'ma':
        short = y.rolling(short_window).apply(lambda x: EMA(x, 0.1))
        long = y.rolling(long_window).apply(lambda x: EMA(x, 0.1))
        UpwardTrend = (short > long)
        Decision = UpwardTrend.rolling(2).apply(lambda x: position(x))
        return Decision
    
    elif method == 'rsi':
        RSIndex = y.diff().rolling(long_window).apply(lambda x: RSI(x))
        SellSignal = (RSIndex >= 70)*(-1.0)
        BuySignal = (RSIndex <= 30)*(1.0)
        WaitSignal = ((RSIndex > 30) & (RSIndex < 70))*(1.0)
        return pd.DataFrame([y, RSIndex, SellSignal, BuySignal, WaitSignal])
        
    elif method == 'macd':
        func = lambda x: EMA(x, 0.1)
    elif method == 'cvalue':
        func = lambda x: cValue(x)
        
    BaseLine = y.rolling(short_window).apply(func=func) - y.rolling(long_window).apply(func=func)
    SignalLine = BaseLine.rolling(signal_window).apply(lambda x: EMA(x, 0.1))
    UpwardTrend = BaseLine > SignalLine
    Decision = UpwardTrend.rolling(2).apply(lambda x: position(x))
    return Decision

def MaxReturn(y: pd.Series):
     y_ret = y.pct_change()
     max_return = (1 + y_ret[y_ret>0]).prod() - 1
     return max_return

In [76]:
Returns = data.pct_change()
Returns.tail()

,NEM.US,AU.US,RGLD.US
Date,,,
2025-01-21,0.01,0.02,0.01
2025-01-22,-0.01,0.00,-0.01
2025-01-23,0.00,-0.00,-0.00
2025-01-24,0.01,0.01,0.01
2025-01-27,-0.01,-0.01,-0.01


In [77]:
y = data['NEM.US']
temp_df = Decision(y, method='rsi').T
temp_df.columns = ['y', 'RSI', 'SellSignal', 'BuySignal', 'WaitSignal']
temp_df = temp_df.dropna()
temp_df.head(20)

,y,RSI,SellSignal,BuySignal,WaitSignal
Date,,,,,
2020-02-10,44.21,54.77,-0.00,0.00,1.00
2020-02-11,44.18,57.58,-0.00,0.00,1.00
2020-02-12,43.81,53.17,-0.00,0.00,1.00
2020-02-13,43.95,53.71,-0.00,0.00,1.00
2020-02-14,43.99,62.04,-0.00,0.00,1.00
2020-02-18,45.53,71.11,-1.00,0.00,0.00
2020-02-19,46.03,70.58,-1.00,0.00,0.00
2020-02-20,48.08,75.20,-1.00,0.00,0.00
2020-02-21,49.47,77.89,-1.00,0.00,0.00


In [51]:
MaxReturn = data.apply(lambda y: MaxReturn(y))
MaxReturn.name = 'MaxReturn'
MaxReturn

NEM.US      37045.10
AU.US     2655583.86
RGLD.US     18011.45
Name: MaxReturn, dtype: float64

In [52]:
MADecision = data.apply(lambda y: Decision(y, method='ma', short_window=10, long_window=22))
MAReturn = (1 + Returns[MADecision == 1.0]).prod() - 1
MAReturn.name = 'MAReturn'
MAReturn

NEM.US    1.04
AU.US     1.07
RGLD.US   1.75
Name: MAReturn, dtype: float64

In [53]:
MACDDecision = data.apply(lambda y: Decision(y, method='macd', short_window=12, long_window=26))
MACDReturn = (1 + Returns[MACDDecision == 1.0]).prod() - 1
MACDReturn.name = 'MACDReturn'
MACDReturn

NEM.US    1.62
AU.US     3.64
RGLD.US   2.10
Name: MACDReturn, dtype: float64

In [38]:
cValueDecision = data.apply(lambda y: Decision(y, method='cvalue', short_window=12, long_window=26))
cValueReturn = (1 + Returns[cValueDecision == 1.0]).prod() - 1
cValueReturn.name = 'cValueReturn'
cValueReturn

NEM.US     1.792647
AU.US      5.980835
RGLD.US    2.104736
Name: cValueReturn, dtype: float64

In [46]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
100*pd.DataFrame([MaxReturn, MAReturn, MACDReturn, cValueReturn], dtype=float)

,NEM.US,AU.US,RGLD.US
MaxReturn,3704509.90,265558386.05,1801144.66
MAReturn,103.70,107.14,174.62
MACDReturn,162.29,363.93,209.84
cValueReturn,179.26,598.08,210.47


In [10]:
for symbol in data.columns:
    y = data.loc[:, symbol]
    Returns = y.pct_change()
    
    MaxReturn = (1 + Returns[Returns>0]).prod() - 1
    
    # Strategia porównywania MA
    MAshort = y.rolling(10).apply(lambda x: EMA(x, 0.1))
    MAlong = y.rolling(22).apply(lambda x: EMA(x, 0.1))
    MAUpwardTrend = (MAshort > MAlong)
    MADecision = MAUpwardTrend.rolling(2).apply(lambda x: position(x))

    MAReturn = (1 + Returns[MADecision == 1.0]).prod() - 1
    
    # Strategia MACD
    MACDDecision = data.apply(lambda y: Decision(y, 'MACD'))[symbol]
    MACDReturn = (1 + Returns[MACDDecision == 1.0]).prod() - 1
    
    # Strategia cValue
    cValueDecision = data.apply(lambda y: Decision(y, 'cValue'))[symbol]
    cValueReturn = (1 + Returns[cValueDecision == 1.0]).prod() - 1
    
    # Strategia łączona
    CombinedDecision = (cValueDecision == 1.0) & (MACDDecision == 1.0)
    CombinedReturn = (1 + Returns[CombinedDecision == 1.0]).prod() - 1
    
    # Podsumowanie
    print(f"{symbol}:\n\tSkumulowany zwrot ze strategii opartej na porównywaniu MA: {MAReturn:.2%}")
    print(f"\tSkumulowany zwrot ze strategii opartej na MACD: {MACDReturn:.2%}")
    print(f"\tSkumulowany zwrot ze strategii opartej na cValue: {cValueReturn:.2%}")
    print(f"\tSkumulowany zwrot ze strategii połączonej: {CombinedReturn:.2%}")
    print(f"\tSkumulowany maksymalny zwrot: {MaxReturn:.2%}")

NEM.US:
	Skumulowany zwrot ze strategii opartej na porównywaniu MA: 103.70%
	Skumulowany zwrot ze strategii opartej na MACD: 162.29%
	Skumulowany zwrot ze strategii opartej na cValue: 179.26%
	Skumulowany zwrot ze strategii połączonej: 184.70%
	Skumulowany maksymalny zwrot: 3704509.90%
AU.US:
	Skumulowany zwrot ze strategii opartej na porównywaniu MA: 107.14%
	Skumulowany zwrot ze strategii opartej na MACD: 363.93%
	Skumulowany zwrot ze strategii opartej na cValue: 598.08%
	Skumulowany zwrot ze strategii połączonej: 348.50%
	Skumulowany maksymalny zwrot: 265558386.05%
RGLD.US:
	Skumulowany zwrot ze strategii opartej na porównywaniu MA: 174.62%
	Skumulowany zwrot ze strategii opartej na MACD: 209.84%
	Skumulowany zwrot ze strategii opartej na cValue: 210.47%
	Skumulowany zwrot ze strategii połączonej: 168.52%
	Skumulowany maksymalny zwrot: 1801144.66%
